In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [4]:
df = pd.read_csv("/content/Fake-News-Classification_NLP/News_dataset.csv")
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [6]:
df.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
df = df.dropna()

In [8]:
df.isna().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [9]:
df.reset_index(inplace=True)
df.head()

,index,id,title,author,text,label
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [12]:
df = df.drop(['id','text','author'],axis = 1)
df.head()


,index,title,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",0
2,2,Why the Truth Might Get You Fired,1
3,3,15 Civilians Killed In Single US Airstrike Hav...,1
4,4,Iranian woman jailed for fictional unpublished...,1


In [10]:
sample_data = 'The quick brown fox jumps over the lazy dog'
sample_data = sample_data.split()
sample_data

['The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']

In [11]:

sample_data = [data.lower() for data in sample_data]
sample_data

['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']

In [15]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [16]:
# Remove Stopwords
stopwords = stopwords.words('english')
print(stopwords[0:10])
print(len(stopwords))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]
179


In [17]:
sample_data = [data for data in sample_data if data not in stopwords]
print(sample_data)
len(sample_data)

['quick', 'brown', 'fox', 'jumps', 'lazy', 'dog']


6

In [18]:
# stemming

In [19]:
ps = PorterStemmer()
sample_data_stemming =[ps.stem(data) for data in sample_data]
print(sample_data_stemming)

['quick', 'brown', 'fox', 'jump', 'lazi', 'dog']


In [20]:
# lemmatization

In [21]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [22]:
lm = WordNetLemmatizer()
sample_data_lema =[lm.lemmatize(data) for data in sample_data]
print(sample_data_lema)

['quick', 'brown', 'fox', 'jump', 'lazy', 'dog']


In [23]:
df['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [24]:
lm  =WordNetLemmatizer()
corpus = []
for i in range(len(df)):
  review = re.sub('^a-zA-Z0-9',' ',df['title'][i])
  review = review.lower()
  review = review.split()
  review = [lm.lemmatize(x) for x in review if x not in stopwords]
  review = " ".join(review)
  corpus.append(review)



In [25]:
len(corpus)

18285

In [26]:
df['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [28]:
corpus[0]

'house dem aide: didn’t even see comey’s letter jason chaffetz tweeted'

In [29]:
# vectorization(convert text data into vector)
tf = TfidfVectorizer()
x = tf.fit_transform(corpus).toarray()
x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [30]:
y =df['label']
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [31]:
# splitting data train & test data
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=10,stratify = y)

In [32]:
len(x_train),len(y_train)

(12799, 12799)

In [33]:
len(x_test),len(y_test)

(5486, 5486)

In [34]:
# model building
rf = RandomForestClassifier()
rf.fit(x_train,y_train)

RandomForestClassifier()

## model evaluvation

In [35]:
y_pred = rf.predict(x_test)


In [36]:
accuracy_score_ = accuracy_score(y_test,y_pred)
accuracy_score_

0.9363835216915786

In [39]:
class Evaluation:
  def __init__(self,model,x_train,x_test,y_train,y_test):
    self.model = model
    self.x_train = x_train
    self.x_test = x_test
    self.y_train = y_train
    self.y_test = y_test
  def train_evaluation(self):
    y_pred_train = self.model.predict(self.x_train)

    acc_scr_train = accuracy_score(self.y_train,y_pred_train)
    print("Accuracy score on training data:",acc_scr_train)
    print()

    con_mat_train = confusion_matrix(self.y_train,y_pred_train)
    print("Confusion matrix on training data :\n",con_mat_train)
    print()

    clas_report_train = classification_report(self.y_train,y_pred_train)
    print("Classifier report on training data:\n",clas_report_train)

  def test_evaluation(self):
    y_pred_test = self.model.predict(self.x_test)

    acc_scr_test = accuracy_score(self.y_test,y_pred_test)
    print("Accuracy score on testing data:",acc_scr_test)
    print()

    con_mat_test = confusion_matrix(self.y_test,y_pred_test)
    print("Confusion matrix on testing data :\n",con_mat_test)
    print()

    clas_report_test = classification_report(self.y_test,y_pred_test)
    print("Classifier report on testing data:\n",clas_report_test)


In [40]:
 Evaluation(rf,x_train,x_test,y_train,y_test).train_evaluation()

Accuracy score on training data: 1.0

Confusion matrix on training data :
 [[7252    0]
 [   0 5547]]

Classifier report on training data:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      7252
           1       1.00      1.00      1.00      5547

    accuracy                           1.00     12799
   macro avg       1.00      1.00      1.00     12799
weighted avg       1.00      1.00      1.00     12799



In [41]:
Evaluation(rf,x_train,x_test,y_train,y_test).test_evaluation()

Accuracy score on testing data: 0.9363835216915786

Confusion matrix on testing data :
 [[2819  290]
 [  59 2318]]

Classifier report on testing data:
               precision    recall  f1-score   support

           0       0.98      0.91      0.94      3109
           1       0.89      0.98      0.93      2377

    accuracy                           0.94      5486
   macro avg       0.93      0.94      0.94      5486
weighted avg       0.94      0.94      0.94      5486



prediction pipeline

In [42]:
class preprocessing:
  def __init__(self,data):
    self.data = data

  def text_preprocessing_user(self):
    lm  =WordNetLemmatizer()
    pred_data = [self.data]
    preprocess_data = []
    for data in pred_data:
      review = re.sub('^a-zA-Z0-9',' ',data)
      review = review.lower()
      review = review.split()
      review = [lm.lemmatize(x) for x in review if x not in stopwords]
      review = " ".join(review)
      preprocess_data.append(review)
    return preprocess_data



In [43]:
df['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [44]:
data = 'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'
preprocessing(data).text_preprocessing_user()

['flynn: hillary clinton, big woman campus - breitbart']

In [55]:
class prediction:
  def __init__(self,pred_data,model):
    self.pred_data=pred_data
    self.model = model

  def prediction_model(self):
    preprocess_data = preprocessing(self.pred_data).text_preprocessing_user()
    data = tf.transform(preprocess_data)
    prediction = self.model.predict(data)

    if prediction[0]==0:
      return "The news is Fake"
    else:
      return "The news is real"



In [57]:
data = 'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'
prediction(data,rf).prediction_model()

'The news is Fake'